<a href="https://colab.research.google.com/github/ejlee127/sound-learning/blob/main/train_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Fetched 252 kB in 2s (149 kB/s)
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-11-04 18:29:40--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.40MB/s    in 0.6s    

2020-11-04 18:29:41 (1.40 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
import numpy as np
import json
# Read in data from S3 Buckets
from pyspark import SparkFiles

def pulljsonFiles(filename):
  json_url="https://billdatavizclass.s3.amazonaws.com/" + filename
  spark.sparkContext.addFile(json_url)
  data= spark.read.option('multiline','true').json(SparkFiles.get(filename))

  # return the data from the file
  inputsTmp = data.select("mfcc").collect()
  inputs = inputsTmp[0][0] #  
  targetsTmp = data.select("classID").collect()
  targets = targetsTmp[0][0]

  return inputs,targets



In [5]:
def createSet(inputs,targets,currentSet=[], currentTarget=[]):

  for i in range(len(inputs)):
      currentSet.append(inputs[i])
      currentTarget.append(targets[i])
  return currentSet, currentTarget

In [6]:
fileList = ["data1.json","data2.json","data3.json","data4.json","data5.json","data6.json","data7.json","data8.json","data9.json","data10.json"]
testList = ["data10.json"]
X_test = []
X_train = []
y_test = []
y_train = []

for file in fileList:
  inputs, targets = pulljsonFiles(file)
  print(np.shape(inputs))
  if file in testList: X_test, y_test = createSet(inputs,targets,X_test, y_test)
  else: X_train, y_train = createSet(inputs,targets,X_train, y_train)
  print(f"done with file: {file}")
print("shape",np.shape(X_train))
print("shape",np.shape(X_test))
X_test = np.array(X_test)
X_train = np.array(X_train)
y_test = np.array(y_test)
y_train = np.array(y_train)

(3106, 44, 13)
done with file: data1.json
(3076, 44, 13)
done with file: data2.json
(3337, 44, 13)
done with file: data3.json
(3454, 44, 13)
done with file: data4.json
(3248, 44, 13)
done with file: data5.json
(2901, 44, 13)
done with file: data6.json
(3010, 44, 13)
done with file: data7.json
(2809, 44, 13)
done with file: data8.json
(2910, 44, 13)
done with file: data9.json
(2974, 44, 13)
done with file: data10.json
shape (27851, 44, 13)
shape (2974, 44, 13)


In [7]:
print(X_train.shape[0],X_train.shape[1], X_train.shape[2])
print(y_train.shape[0], y_train.shape[1])
print(X_test.shape[0],X_test.shape[1], X_test.shape[2])
print(y_test.shape[0], y_test.shape[1])


27851 44 13
27851 1
2974 44 13
2974 1


In [26]:
print(X_train)

[[[-6.01794495e+02  4.11970062e+01  3.20108414e+01 ... -2.74509740e+00
   -2.52226734e+00 -1.33190060e+00]
  [-5.97776184e+02  4.62298927e+01  3.53872452e+01 ... -4.28198367e-01
   -1.25637817e+00 -1.31263065e+00]
  [-5.96598877e+02  4.73582573e+01  3.54797668e+01 ...  1.44725585e+00
    5.12672424e-01  8.63579214e-01]
  ...
  [-5.93491577e+02  4.96914597e+01  3.38171310e+01 ...  3.66332054e+00
    4.71112823e+00  5.57440090e+00]
  [-5.93208069e+02  5.11510773e+01  3.72942772e+01 ...  4.22459555e+00
    2.99432063e+00  4.03167295e+00]
  [-5.92189941e+02  5.24554749e+01  3.84227219e+01 ...  5.71697521e+00
    2.21275377e+00  2.35637856e+00]]

 [[-5.10668243e+02  2.89186211e+01  2.65940552e+01 ...  4.19182968e+00
    2.47202516e+00  1.18620515e+00]
  [-4.61764465e+02  8.29992065e+01  4.85441704e+01 ...  4.16392565e-01
   -5.59840775e+00 -2.03919435e+00]
  [-3.40975128e+02  1.73004517e+02  7.34781647e+00 ... -1.29536161e+01
   -1.49046726e+01 -8.55358028e+00]
  ...
  [-5.15874634e+02  2.1

In [12]:
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [13]:
# build the network architecture
model = keras.Sequential([
    # input layer
    keras.layers.Flatten(input_shape=(X_train.shape[1], X_train.shape[2])),
    
    # lst hidden layer
    keras.layers.Dense(512, activation="relu"),
    
    # 2nd hidden layer
    keras.layers.Dense(256, activation="relu"),
    
    # 3rd hidden layer
    keras.layers.Dense(64, activation="relu"),
    
    # output layer
    keras.layers.Dense(10, activation="softmax")
])

In [16]:
# compile network : example
# optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer='adam',
             loss = "sparse_categorical_crossentropy",
             metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 572)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               293376    
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_6 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_7 (Dense)              (None, 10)                650       
Total params: 441,802
Trainable params: 441,802
Non-trainable params: 0
_________________________________________________________________


In [17]:
# train network
model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)

## overfitting -- identify

Epoch 1/50
871/871 [==============================] - 4s 5ms/step - loss: 2.7805 - accuracy: 0.4487 - val_loss: 1.9749 - val_accuracy: 0.4425
Epoch 2/50
871/871 [==============================] - 4s 5ms/step - loss: 1.1404 - accuracy: 0.6143 - val_loss: 2.0571 - val_accuracy: 0.4839
Epoch 3/50
871/871 [==============================] - 4s 5ms/step - loss: 0.9355 - accuracy: 0.6854 - val_loss: 2.1196 - val_accuracy: 0.5013
Epoch 4/50
871/871 [==============================] - 4s 5ms/step - loss: 0.8083 - accuracy: 0.7318 - val_loss: 2.4428 - val_accuracy: 0.5313
Epoch 5/50
871/871 [==============================] - 4s 5ms/step - loss: 0.7390 - accuracy: 0.7571 - val_loss: 2.7087 - val_accuracy: 0.4637
Epoch 6/50
871/871 [==============================] - 4s 5ms/step - loss: 0.6672 - accuracy: 0.7815 - val_loss: 2.8619 - val_accuracy: 0.4398
Epoch 7/50
871/871 [==============================] - 4s 5ms/step - loss: 0.6086 - accuracy: 0.7974 - val_loss: 3.4350 - val_accuracy: 0.4129
Epoch 

In [19]:
# Save the model
model.save("sound_model_trained.h5")

In [18]:
model_loss, model_accuracy = model.evaluate(
    X_test, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

93/93 - 0s - loss: 7.2087 - accuracy: 0.4516
Normal Neural Network - Loss: 7.208740234375, Accuracy: 0.45158037543296814
